In [1]:
from pandas import read_csv, DataFrame, Index

In [2]:
exchanges = ['Huobi', 'Deribit', 'Bybit', 'Bitmex', 'OKX', 'Binance', 'Bitget', 'Kraken']

In [3]:
huobi_data = read_csv('data/btc inverse perps/Huobi_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
huobi_data.name = 'Huobi'

binance_data = read_csv('data/btc inverse perps/Binance_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
binance_data.name = 'Binance'

deribit_data = read_csv('data/btc inverse perps/Deribit_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
deribit_data = deribit_data.iloc[2:,:]
deribit_data.name = 'Deribit'

bybit_data = read_csv('data/btc inverse perps/Bybit_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
bybit_data = bybit_data.iloc[2:,:]
bybit_data.name = 'Bybit'

bitget_data = read_csv('data/btc inverse perps/Bitget_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
bitget_data.name = 'Bitget'

bitmex_data = read_csv('data/btc inverse perps/Bitmex_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
bitmex_data = bitmex_data.iloc[2:,:]
bitmex_data.name = 'Bitmex'

okx_data = read_csv('data/btc inverse perps/OKX_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
okx_data = okx_data.iloc[2:,:]
okx_data.name = 'OKX'

kraken_data = read_csv('data/btc inverse perps/Kraken_1d.csv', parse_dates=['timestamp'], index_col='timestamp')
kraken_data.name = 'Kraken'

In [4]:
dfs_2020Q2_2023Q1 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data]
dfs_2023Q2 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data, bitget_data, kraken_data]
dfs_2023Q3_2023Q4 = [huobi_data, deribit_data, bybit_data, bitmex_data, okx_data, binance_data, bitget_data, kraken_data]

In [5]:
def agg_OI_quarterly(df):
    ret = df['OI Close'].resample('Q').sum()
    ret = ret.to_frame(name='Agg OI').reset_index()
    ret['Quarter'] = ret['timestamp'].dt.to_period('Q')
    ret.name = df.name
    return ret

In [6]:
dfs_2020Q2_2023Q1_quarterly_OI = [agg_OI_quarterly(df) for df in dfs_2020Q2_2023Q1]
dfs_2023Q2_quarterly_OI = [agg_OI_quarterly(df) for df in dfs_2023Q2]
dfs_2023Q3_2023Q4 = [agg_OI_quarterly(df) for df in dfs_2023Q3_2023Q4]

In [7]:
def get_ratio(df, dfs, quarter):
    sum_of_quarter = 0
    this_value = df.loc[df['Quarter'] == quarter]['Agg OI'].values[0]
    for _df in dfs:
        sum_of_quarter += _df.loc[_df['Quarter'] == quarter]['Agg OI'].values[0]
    return df.name, quarter, this_value / sum_of_quarter

In [8]:
quarter_list = [
    '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1',
    '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4'
]

In [9]:
ratios_by_quarter = []

for quarter in quarter_list:
    ratios = []

    if quarter == '2023Q2':
        for df in dfs_2023Q2_quarterly_OI:
            _, _, ratio = get_ratio(df, dfs_2023Q2_quarterly_OI, quarter)
            ratios.append(ratio)
    elif quarter in ['2023Q3', '2023Q4']:
        for df in dfs_2023Q3_2023Q4:
            _, _, ratio = get_ratio(df, dfs_2023Q3_2023Q4, quarter)
            ratios.append(ratio)
    else:
        for df in dfs_2020Q2_2023Q1_quarterly_OI:
            _, _, ratio = get_ratio(df, dfs_2020Q2_2023Q1_quarterly_OI, quarter)
            ratios.append(ratio)

    ratios_by_quarter.append(ratios)

In [10]:
for i in range(len(ratios_by_quarter)):
    if i <= 11:
        ratios_by_quarter[i].insert(5, 0)
        ratios_by_quarter[i].insert(6, 0)
        ratios_by_quarter[i].insert(7, 0)
    if i == 12:
        ratios_by_quarter[i].insert(5, 0)

In [11]:
oi_ratios_by_quarter_and_exchange = DataFrame(
    ratios_by_quarter,
    columns=exchanges
)

oi_ratios_by_quarter_and_exchange = oi_ratios_by_quarter_and_exchange.set_index(Index(quarter_list))
oi_ratios_by_quarter_and_exchange['Checksum'] = oi_ratios_by_quarter_and_exchange.sum(axis=1)

In [12]:
oi_ratios_by_quarter_and_exchange

,Huobi,Deribit,Bybit,Bitmex,OKX,Binance,Bitget,Kraken,Checksum
2020Q2,0.091932,0.074491,0.171595,0.550159,0.111822,0.000000,0.000000,0.000000,1.0
2020Q3,0.153404,0.072746,0.213648,0.467397,0.092805,0.000000,0.000000,0.000000,1.0
2020Q4,0.144935,0.111903,0.361464,0.274520,0.107177,0.000000,0.000000,0.000000,1.0
2021Q1,0.128307,0.139291,0.455804,0.189173,0.087424,0.000000,0.000000,0.000000,1.0
2021Q2,0.125797,0.153031,0.455724,0.175106,0.090341,0.000000,0.000000,0.000000,1.0
2021Q3,0.091755,0.180789,0.413364,0.228944,0.085148,0.000000,0.000000,0.000000,1.0
2021Q4,0.047300,0.193181,0.484304,0.157884,0.117330,0.000000,0.000000,0.000000,1.0
2022Q1,0.048794,0.237229,0.373900,0.187355,0.152722,0.000000,0.000000,0.000000,1.0
2022Q2,0.036651,0.260827,0.323576,0.164361,0.214584,0.000000,0.000000,0.000000,1.0
2022Q3,0.032399,0.263301,0.268561,0.175353,0.260385,0.000000,0.000000,0.000000,1.0


In [13]:
def get_weighted_fr(exchange_data):
    exchange_quarter_weights = dict(oi_ratios_by_quarter_and_exchange[exchange_data.name])

    ret = exchange_data.copy()
    ret = ret.reset_index()
    ret['Quarter'] = ret['timestamp'].dt.to_period('Q')
    ret['Weighted FR'] = ret['FR Close'] * ret['Quarter'].replace(exchange_quarter_weights)
    
    ret = ret.drop(['OI Open', 'OI High', 'OI Low', 'OI Close', 'FR Open', 'FR High', 'FR Low', 'FR Close', 'Quarter'], axis=1)
    ret = ret.set_index('timestamp')

    sum_wfr = ret["Weighted FR"].sum()

    return ret, sum_wfr

In [14]:
huobi_wfr, huobi_wfr_sum = get_weighted_fr(huobi_data)
huobi_wfr

,Open,High,Low,Close,Volume,Weighted FR
timestamp,,,,,,
2020-04-13,6776.0,6964.7,6742.2,6916.1,100048.827253,-0.002067
2020-04-14,6916.2,6984.0,6681.6,6704.8,94522.590938,-0.000379
2020-04-15,6704.7,7176.9,6465.0,7055.9,193494.686859,0.000919
2020-04-16,7056.0,7206.6,6961.3,7046.8,125713.201910,0.000919
2020-04-17,7046.7,7282.8,7019.5,7247.8,118498.150122,0.000919
...,...,...,...,...,...,...
2023-12-27,43058.9,43893.0,42528.0,42553.9,733.294885,0.000636
2023-12-28,42544.3,43217.8,42244.9,42501.6,704.376715,0.001332
2023-12-29,42465.6,44050.0,40409.0,42499.5,2782.342411,0.001297


In [15]:
deribit_wfr, deribit_wfr_sum = get_weighted_fr(deribit_data)
deribit_wfr

,Open,High,Low,Close,Volume,Weighted FR
timestamp,,,,,,
2020-04-01,6334.0,6774.0,6151.5,6662.5,27598.146558,0.000000
2020-04-02,6662.5,7267.0,6615.0,6941.5,33789.541207,0.000000
2020-04-03,6942.0,7058.5,6605.5,6740.0,21935.119849,0.000000
2020-04-04,6740.0,7022.5,6645.0,6778.5,19161.138555,0.000000
2020-04-05,6778.5,7118.5,6670.0,7043.0,17546.976938,0.000000
...,...,...,...,...,...,...
2023-12-27,42480.5,43854.5,42428.0,43105.5,10631.480347,0.011102
2023-12-28,43105.5,43238.5,42159.5,42343.5,15167.908298,0.011449
2023-12-29,42343.5,43159.0,41241.0,41995.0,14935.406162,0.008374


In [16]:
bybit_wfr, bybit_wfr_sum = get_weighted_fr(bybit_data)
bybit_wfr

,Open,High,Low,Close,Volume,Weighted FR
timestamp,,,,,,
2020-04-01,6424.0,6718.5,6155.0,6670.5,124976.924377,-0.006847
2020-04-02,6670.5,7273.5,6571.5,6799.0,185506.163476,0.001716
2020-04-03,6799.0,7060.0,6610.0,6738.5,129353.230502,0.001716
2020-04-04,6738.5,7020.5,6642.0,6871.0,119537.539156,0.001527
2020-04-05,6871.0,6911.0,6661.0,6775.5,77271.068976,0.001201
...,...,...,...,...,...,...
2023-12-27,42552.0,43700.0,42150.0,43469.5,13469.016121,0.001937
2023-12-28,43469.5,43872.0,42296.0,42621.0,12142.951188,0.005037
2023-12-29,42621.0,43176.5,41260.0,42090.5,17448.185327,0.001937


In [17]:
bitmex_wfr, bitmex_wfr_sum = get_weighted_fr(bitmex_data)
bitmex_wfr

,Open,High,Low,Close,Volume,Weighted FR
timestamp,,,,,,
2020-04-01,6425.0,6723.5,6141.5,6671.5,2.207635e+09,-0.029268
2020-04-02,6671.5,7299.0,6571.0,6795.5,2.972606e+09,0.005502
2020-04-03,6795.5,7049.5,6600.0,6738.0,2.140976e+09,-0.023822
2020-04-04,6738.0,7097.0,6600.0,6875.0,1.619523e+09,-0.008307
2020-04-05,6875.0,6920.0,6660.5,6775.5,1.148472e+09,0.000385
...,...,...,...,...,...,...
2023-12-27,42533.0,43728.0,42130.0,43475.0,3.578441e+08,0.003929
2023-12-28,43475.0,43893.5,42307.5,42610.0,3.358834e+08,0.002334
2023-12-29,42610.0,43177.5,41284.0,42080.0,4.314916e+08,0.000922


In [18]:
okx_wfr, okx_wfr_sum = get_weighted_fr(okx_data)
okx_wfr

,Open,High,Low,Close,Volume,Weighted FR
timestamp,,,,,,
2020-04-01,6424.0,6723.0,6151.3,6670.8,77037.0376,-0.002848
2020-04-01,6424.0,6723.0,6151.3,6670.8,77037.0376,-0.002848
2020-04-02,6670.1,7309.1,6576.4,6799.5,93328.3721,-0.000808
2020-04-02,6670.1,7309.1,6576.4,6799.5,93328.3721,-0.000808
2020-04-03,6799.5,7063.9,6607.0,6743.0,67670.6400,0.000807
...,...,...,...,...,...,...
2023-12-27,42527.2,43715.0,42125.5,43457.0,17455.4669,0.007794
2023-12-28,43457.1,43861.6,42297.7,42611.1,18974.1377,0.003892
2023-12-29,42611.2,43168.0,41250.1,42082.9,28280.1150,0.002712


In [19]:
binance_wfr, binance_wfr_sum = get_weighted_fr(binance_data)
binance_wfr

,Open,High,Low,Close,Volume,Weighted FR
timestamp,,,,,,
2023-08-01,29216.2,29732.0,28544.4,29698.2,79030.740710,0.001764
2023-08-02,29698.1,30064.8,28910.2,29164.0,79844.199800,0.001764
2023-08-03,29164.1,29417.0,28950.0,29182.0,30858.524600,0.001764
2023-08-04,29182.0,29312.1,28690.0,29078.8,27892.646751,0.001764
2023-08-05,29078.7,29115.6,28955.7,29057.9,7271.133066,0.001764
...,...,...,...,...,...,...
2023-12-27,42546.5,43718.6,42156.4,43484.1,32993.501700,0.002270
2023-12-28,43484.2,43869.3,42322.2,42615.3,41161.351849,0.004266
2023-12-29,42615.4,43181.4,41275.0,42101.8,49031.376533,0.002884


In [20]:
bitget_wfr, bitget_wfr_sum = get_weighted_fr(bitget_data)
bitget_wfr

,Open,High,Low,Close,Volume,Weighted FR
timestamp,,,,,,
2023-04-22,27242.5,27857.5,27118.0,27790.0,1598.557,0.000000
2023-04-23,27790.0,27790.5,27305.0,27569.0,1498.882,0.000000
2023-04-24,27569.0,27977.0,26942.5,27487.5,2514.170,0.000000
2023-04-25,27487.5,28371.0,27176.5,28274.5,1492.539,0.000000
2023-04-26,28274.5,30022.5,27200.0,28403.0,6952.085,0.000000
...,...,...,...,...,...,...
2023-12-29,42594.3,43158.0,41267.5,42058.7,2807.240,0.001273
2023-12-30,42058.7,42618.8,41538.0,42152.4,1822.208,0.002613
2023-12-30,42058.7,42618.8,41538.0,42152.4,1822.208,0.002613


In [21]:
kraken_wfr, kraken_wfr_sum = get_weighted_fr(kraken_data)
kraken_wfr

,Open,High,Low,Close,Volume,Weighted FR
timestamp,,,,,,
2023-05-20,26876.0,27147.5,26831.0,27103.5,7877846.0,-0.000012
2023-05-21,27103.5,27280.0,26668.0,26762.5,14491745.0,0.000008
2023-05-22,26762.5,27114.0,26543.5,26860.0,19565298.0,-0.000009
2023-05-23,26860.0,27502.0,26808.0,27237.0,19913001.0,0.000028
2023-05-24,27237.0,27237.0,26080.5,26338.5,22141046.0,0.000022
...,...,...,...,...,...,...
2023-12-29,42654.0,43190.0,41293.5,42065.0,21865808.0,0.000015
2023-12-30,42065.0,42636.5,41502.0,42166.5,8836551.0,0.000025
2023-12-30,42065.0,42636.5,41502.0,42166.5,8836551.0,0.000025


In [23]:
res = DataFrame([
    [
        huobi_wfr_sum,
        deribit_wfr_sum,
        bybit_wfr_sum,
        bitmex_wfr_sum,
        okx_wfr_sum,
        binance_wfr_sum,
        bitget_wfr_sum,
        kraken_wfr_sum
    ]
])

res.columns = exchanges
res.index = ['Weighted Funding Rate Sum']
res.T

,Weighted Funding Rate Sum
Huobi,1.266591
Deribit,1.126411
Bybit,7.353444
Bitmex,1.557681
OKX,1.034676
Binance,0.219997
Bitget,0.237186
Kraken,0.001877
